In [ ]:
# This notebook can be used to plot our hadronic mass measurements, presented in the appendix.

In [1]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from ipynb.fs.full.CoefficientsCalcPlus import GetCoefficientsFlux
from ipynb.fs.full.CoefficientsCalcPlus import get_normalization
import sklearn.linear_model
import scipy
import warnings; warnings.simplefilter('ignore')
import os
from matplotlib.pyplot import figure
import random as rnd
import matplotlib.ticker as ticker

%jsroot on

Welcome to JupyROOT 6.30/04


# Run these blocks once to prepare histograms file

In [ ]:
files = ["/eos/home-a/amgruber/SWAN_projects/XSec_ROOT/TTreeMerges/proj"+str(i+1)+"_merged.root" for i in range(58)]
# Change this to your preferred location; Merged trees can be produced using MergeTTrees.ipynb

In [ ]:
coeffs, std = GetCoefficientsFlux(0.75,0.07,1e-12,model=sklearn.linear_model.Ridge,years=1)
coeffs = np.array(coeffs)
norm = get_normalization(coeffs)
years = 20
res = []  # [toy][omega bin]

normalization = open("norms.txt", "r").read().split("\n")[:-1]
normalization = np.array([float(n) for n in normalization])

total_Nbins = int(16000)
bin_min = -int(8000)
bin_max = int(8000)

Q2_reco = "((pmu_4mom.Px()**2 + pmu_4mom.Py()**2 + (750-pmu_4mom.Pz())**2)-(750-ELep)**2)"

nucleon_mass = str((938.272 + 939.565)/2.0)
W_reco = "sqrt("+nucleon_mass+"**2 + 2*"+nucleon_mass+"*(750-ELep)-"+Q2_reco+")"

modes = {"CCQE": "Mode == 1",
         "2p2h": "Mode == 2",
         "RES": "Mode == 11 || Mode == 12 || Mode == 13",
         "Other": "Mode != 1 && Mode != 2 && Mode != 11 && Mode != 12 && Mode != 13"}

reco_hist = ROOT.TH1F("reco", "reco", total_Nbins, bin_min, bin_max)
mode_hists = [ROOT.TH1F(mode, mode, total_Nbins, bin_min, bin_max) for mode in modes]

modes_arrays = [[] for _ in modes]
bins = 58

for i in range(bins):
    print(i)
    file = ROOT.TFile.Open(files[i], "read")
    t = file.Get("FlatTree_VARS")
    
    reco_added = ROOT.TH1F(f"reco_added{i}", "added", total_Nbins, bin_min, bin_max)
    modes_added = [ROOT.TH1F(f"{mode}_added{i}", "added", total_Nbins, bin_min, bin_max) for mode in modes]
    
    t.Project(f"reco_added{i}", W_reco)  # reco
    
    scale_factor = years * normalization[i] / t.GetEntries()
    reco_added.Scale(scale_factor)
    
    for mode, hist in zip(modes.keys(), modes_added):
        t.Project(hist.GetName(), W_reco, modes[mode])
        hist.Scale(scale_factor)
    
    # Properly set bin errors
    for j in range(1, total_Nbins + 1):
        bin_content = reco_added.GetBinContent(j)
        reco_added.SetBinError(j, np.sqrt(bin_content))

    reco_hist.Add(reco_added, coeffs[i])
    
    for mode_hist, mode_added in zip(mode_hists, modes_added):
        mode_hist.Add(mode_added, coeffs[i])
    
    file.Close()

output_file = ROOT.TFile("output_histograms_W.root", "RECREATE")
reco_hist.Write()
for mode_hist in mode_hists:
    mode_hist.Write()
output_file.Close()

# Conversion to numpy arrays

In [ ]:
# Open the ROOT file
file = ROOT.TFile.Open("output_histograms_W.root", "READ")
if not file or file.IsZombie():
    raise RuntimeError("Error: Cannot open file output_histograms_W.root")
    
# Load histograms
reco_hist = file.Get("reco")
if not reco_hist:
    raise RuntimeError("Error: reco histogram not found!")

# Define the mode names and load their histograms
mode_names = ["CCQE", "2p2h", "RES", "Other"]
mode_hists = {mode: file.Get(mode) for mode in mode_names}

for mode, hist in mode_hists.items():
    if not hist:
        raise RuntimeError(f"Error: {mode} histogram not found!")

# Function to extract histogram data to numpy arrays
def hist_to_numpy(hist):
    bins = hist.GetNbinsX()
    bin_edges = np.array([hist.GetBinLowEdge(i + 1) for i in range(bins + 1)])
    bin_values = np.array([hist.GetBinContent(i + 1) for i in range(bins)])
    bin_errors = np.array([hist.GetBinError(i + 1) for i in range(bins)])
    return bin_edges, bin_values, bin_errors

# Save the reco histogram
reco_edges, reco_values, reco_errors = hist_to_numpy(reco_hist)
np.savez("reco_hist.npz", edges=reco_edges, values=reco_values, errors=reco_errors)

# Save the mode histograms
for mode, hist in mode_hists.items():
    edges, values, errors = hist_to_numpy(hist)
    np.savez(f"{mode}_hist.npz", edges=edges, values=values, errors=errors)

print("Histograms have been successfully saved to NumPy arrays.")


# Plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set global plot styles
plt.rcParams['text.usetex'] = True
plt.rcParams['xtick.labelsize'] = 32
plt.rcParams['ytick.labelsize'] = 32
fontsize = 36

# Define the rebinning and scaling function
def rebin_and_scale(values, edges, errors, scale_factor=1e-4, rebin_factor=20, conversion_factor=1e-3):
    """
    Rebins, scales, and converts histogram data.
    """
    rebinned_values = np.add.reduceat(values, np.arange(0, len(values), rebin_factor))
    rebinned_errors = np.sqrt(np.add.reduceat(errors ** 2, np.arange(0, len(errors), rebin_factor)))
    rebinned_edges = edges[::rebin_factor] * conversion_factor
    rebinned_values *= scale_factor
    rebinned_errors *= scale_factor
    return rebinned_edges, rebinned_values, rebinned_errors

# Load histograms
histograms = {
    "Reco": np.load("reco_hist.npz"),
    "CCQE": np.load("CCQE_hist.npz"),
    "2p2h": np.load("2p2h_hist.npz"),
    "RES": np.load("RES_hist.npz"),
    "Other": np.load("Other_hist.npz")
}

# Define colors and styles
styles = {
    "Reco": {"color": "mediumvioletred", "linestyle": "-", "marker": "o", "markersize": 5, "label": "Reco \n (Stat Unc)"},
    "CCQE": {"color": "blue", "linestyle": "--", "label": "CCQE"},
    "2p2h": {"color": "orange", "linestyle": "--", "label": "2p2h"},
    "RES": {"color": "green", "linestyle": "--", "label": "RES"},
    "Other": {"color": "red", "linestyle": "--", "label": "Other"},
    "Total": {"color": "black", "linestyle": "--", "label": "Total"}
}

# Create the total histogram as the sum of all modes
total_edges, total_values, total_errors = None, None, None
for mode, data in histograms.items():
    edges, values, errors = data["edges"], data["values"], data["errors"]
    rebinned_edges, rebinned_values, rebinned_errors = rebin_and_scale(values, edges, errors)
    
    if total_values is None:
        total_values = rebinned_values.copy()
        total_edges = rebinned_edges
        total_errors = rebinned_errors.copy()
    else:
        total_values += rebinned_values
        total_errors = np.sqrt(total_errors**2 + rebinned_errors**2)

# Plotting
plt.figure(figsize=(12, 8),dpi=300)
plt.grid(True, linestyle='--', alpha=0.7)

# Plot individual histograms
reco_rebinned_edges, reco_rebinned_values, reco_rebinned_errors = None, None, None
for mode, data in histograms.items():
    edges, values, errors = data["edges"], data["values"], data["errors"]
    rebinned_edges, rebinned_values, rebinned_errors = rebin_and_scale(values, edges, errors)
    style = styles[mode]
    
    if mode == "Reco":
        # Plot with error bars for the Reco histogram
        plt.errorbar(
            rebinned_edges[:-1], rebinned_values, yerr=rebinned_errors,
            fmt=style["marker"], color=style["color"], label=style["label"],
            capsize=3, elinewidth=1.5
        )
        # Store rebinned Reco for dashed histogram
        reco_rebinned_edges, reco_rebinned_values = rebinned_edges, rebinned_values
    else:
        plt.step(rebinned_edges[:-1], rebinned_values, where='post',
                 color=style["color"], linestyle=style["linestyle"], label=style["label"])

# Plot the **Reco** histogram with a dashed histogram style
plt.step(
    reco_rebinned_edges[:-1], reco_rebinned_values, 
    where='post', linestyle="--", color="black", label="Total"
)

# Formatting
plt.xlabel(r"$W_{\mathrm{reco}}\ [\mathrm{GeV}/c^2]$", fontsize=fontsize)
plt.ylabel("Events [a.u.]", fontsize=fontsize)
plt.xlim(0, 2.2)  # Converted to GeV
plt.ylim(-0.3, 5.7)
plt.legend(fontsize=fontsize-6, loc='upper right')
plt.tight_layout()
plt.show()
